## Create Wav2Vec2CTCTokenizer

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [2]:
all_data = load_dataset('csv',data_files='./order_speech_ko1000.csv',split='train')

Using custom data configuration default-722dfaf1004ac490


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\AI_server\.cache\huggingface\datasets\csv\default-722dfaf1004ac490\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [3]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [4]:
remove_spectial_char_data = all_data.map(remove_special_characters)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [5]:
df = pd.DataFrame(remove_spectial_char_data)
df.head()

,src,text
0,script1_g_0044-6001-01-01-KSM-F-05-A.wav,보고 있는 영상 정지시켜 줘
1,script1_g_0044-6002-01-01-KSM-F-05-A.wav,다음 주까지 날씨가 어때
2,script1_g_0044-6003-01-01-KSM-F-05-A.wav,나 대신 점등해 줘
3,script1_g_0044-6004-01-01-KSM-F-05-A.wav,이번 주 대체로 흐린지 궁금해
4,script1_g_0044-6005-01-01-KSM-F-05-A.wav,지금 당장 취침 등 꺼 줘


In [6]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [7]:
char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
char_vocab

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})

In [9]:
vocab_list = list(set(char_vocab["vocab"][0]))

In [10]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'미': 0,
 '량': 1,
 '소': 2,
 '핵': 3,
 '말': 4,
 '야': 5,
 '인': 6,
 '줬': 7,
 '풀': 8,
 '뒤': 9,
 '곱': 10,
 '너': 11,
 '은': 12,
 '랑': 13,
 '류': 14,
 '약': 15,
 '놨': 16,
 '겨': 17,
 '존': 18,
 '택': 19,
 '즘': 20,
 '베': 21,
 '른': 22,
 '빛': 23,
 '큼': 24,
 '력': 25,
 '엘': 26,
 '감': 27,
 '와': 28,
 '복': 29,
 '괜': 30,
 '면': 31,
 '봐': 32,
 '록': 33,
 '났': 34,
 '개': 35,
 '마': 36,
 '작': 37,
 '튼': 38,
 '추': 39,
 '역': 40,
 '기': 41,
 '환': 42,
 '목': 43,
 '못': 44,
 '료': 45,
 '혹': 46,
 '토': 47,
 '되': 48,
 '회': 49,
 '찾': 50,
 '낭': 51,
 '먼': 52,
 '듣': 53,
 '예': 54,
 '밤': 55,
 '염': 56,
 '늘': 57,
 '둡': 58,
 '농': 59,
 '큰': 60,
 '졸': 61,
 '셔': 62,
 '확': 63,
 '픈': 64,
 '닫': 65,
 '빙': 66,
 '니': 67,
 '단': 68,
 '나': 69,
 '알': 70,
 '란': 71,
 '연': 72,
 '취': 73,
 '어': 74,
 '맞': 75,
 '집': 76,
 '떤': 77,
 '랐': 78,
 '시': 79,
 '절': 80,
 '찮': 81,
 '느': 82,
 '며': 83,
 '케': 84,
 '열': 85,
 '망': 86,
 '곳': 87,
 '롤': 88,
 '얘': 89,
 '적': 90,
 '낮': 91,
 '먹': 92,
 '돼': 93,
 '꿔': 94,
 '빠': 95,
 '황': 96,
 '혀': 97,
 '의': 98,
 '풍': 99,
 '남': 100,

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

531

In [12]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [13]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [15]:
import torch

In [16]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [17]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [18]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [19]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [20]:
import librosa

In [21]:
def load_audio(batch):
    batch['array'],_ = librosa.load('./dataset/audio/'+batch['src'],sr=16000)
    return batch

In [22]:
audio_data = remove_spectial_char_data.map(load_audio)

  0%|          | 0/1000 [00:00<?, ?ex/s]

## Preprocess Data

In [23]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [24]:
audio_data

Dataset({
    features: ['src', 'text', 'array'],
    num_rows: 1000
})

In [25]:
order_voice = audio_data.map(
    prepare_dataset,
    remove_columns=remove_spectial_char_data.column_names,
    # num_proc=4
)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [26]:
order_voice

Dataset({
    features: ['array', 'input_values', 'labels'],
    num_rows: 1000
})

# Training

## Set-up Trainer

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [32]:
model.freeze_feature_extractor()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [33]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=3,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=order_voice, # train_ds
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [36]:
torch.cuda.is_available()

True

In [38]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: array. If array are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 93


  0%|          | 0/93 [00:00<?, ?it/s]

{'loss': 52.6768, 'learning_rate': 4.8e-06, 'epoch': 0.32}
{'loss': 50.3339, 'learning_rate': 1.0799999999999998e-05, 'epoch': 0.63}
{'loss': 51.2774, 'learning_rate': 1.68e-05, 'epoch': 0.95}
{'loss': 53.6257, 'learning_rate': 2.1599999999999996e-05, 'epoch': 1.29}
{'loss': 48.8287, 'learning_rate': 2.7599999999999997e-05, 'epoch': 1.6}
{'loss': 45.2693, 'learning_rate': 3.36e-05, 'epoch': 1.92}
{'loss': 40.0247, 'learning_rate': 3.9e-05, 'epoch': 2.25}
{'loss': 31.6594, 'learning_rate': 4.4999999999999996e-05, 'epoch': 2.57}
{'loss': 25.5611, 'learning_rate': 5.1e-05, 'epoch': 2.89}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 277.6588, 'train_samples_per_second': 10.805, 'train_steps_per_second': 0.335, 'train_loss': 43.6303466468729, 'epoch': 2.98}


TrainOutput(global_step=93, training_loss=43.6303466468729, metrics={'train_runtime': 277.6588, 'train_samples_per_second': 10.805, 'train_steps_per_second': 0.335, 'train_loss': 43.6303466468729, 'epoch': 2.98})

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.

## Model predict

In [70]:

model(data_collator([order_voice[0]]))

KeyError: 'Indexing with integers is not available when using Python based feature extractors'